In [2]:
import timeit
import time
import math
import numba as nb
import numpy as np
import numexpr as ne
import pandas as pd
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [3]:
%timeit math.sin(0.5)

126 ns ± 4.46 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [4]:
%timeit np.sin(0.5)

1.03 µs ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [5]:
def f_py(x_list, y_list):
    for i in range(len(x_list)):
        y_list[i] = math.sin(x_list[i])

def f_py_listcompr(x_list, y_list):
    return [math.sin(x) for x in x_list]

In [6]:
def f_numpy(x_array, y_array):
    np.sin(x_array, out=y_array)

In [7]:
@nb.njit
def f_numba(x_array, y_array):
    for i in range(x_array.size):
        y_array[i] = np.sin(x_array[i])

@nb.njit(parallel=True)
def f_numba_parall(x_array, y_array):
    for i in nb.prange(x_array.size):
        y_array[i] = np.sin(x_array[i])

@nb.njit
def external_sin(x):
    return np.sin(x)        
        
@nb.njit(parallel=True, fastmath=True, cache=True)
def f_numba_external_parall(x_array, y_array):
    for i in nb.prange(x_array.size):
        y_array[i] = external_sin(x_array[i])

In [8]:
def f_numexpr(x_array, y_array):
    ne.evaluate("sin(x_array)", out=y_array)

In [9]:
#funcs = [f_py, f_py_listcompr, f_numpy, f_numba, f_numba_parall, f_numexpr]
#funcs = [f_numpy, f_numba, f_numba_parall, f_numba_opt_parall, f_numba_opt_parall2, f_numexpr]
funcs = [f_numpy, f_numba, f_numba_external_parall, f_numexpr]
sizes = [1, 10, 100, 200, 500, 1_000, 2_000, 3_000, 5_000, 10_000, 20_000, 50_000, 100_000]#, 200_000, 500_000]

In [10]:
times = np.zeros((len(sizes), len(funcs)))

In [11]:
DTYPE = np.float64

In [12]:
x_nparray = np.linspace(0, 1, num=10, dtype=DTYPE)
y_nparray = np.empty_like(x_nparray)
x_list = x_nparray.tolist()
y_list = y_nparray.tolist()

In [13]:
x_nparray[3], y_nparray[3], id(x_nparray), id(y_nparray), x_nparray.dtype, y_nparray.dtype

(0.3333333333333333,
 0.0,
 140474197314032,
 140474197314224,
 dtype('float64'),
 dtype('float64'))

In [14]:
f_numexpr(x_nparray, y_nparray)

In [15]:
x_nparray[3], y_nparray[3], id(x_nparray), id(y_nparray), x_nparray.dtype, y_nparray.dtype

(0.3333333333333333,
 0.3271946967961522,
 140474197314032,
 140474197314224,
 dtype('float64'),
 dtype('float64'))

In [17]:
id(x_nparray), id(y_nparray)

(140474197314032, 140474197314224)

In [19]:
REPEAT = 7
NUMBER = 1_000

external_sin(0) # 1st time -> compilation

for i, s in enumerate(sizes):
    for j, f in enumerate(funcs):
        x_array = np.linspace(0, 1, num=s, dtype=np.float32)
        y_array = np.empty_like(x_array)     
        if f.__name__.startswith('f_py'):
            x_array = x_array.tolist()
            y_array = y_array.tolist()
        res = timeit.repeat(lambda: f(x_array, y_array), repeat=REPEAT, number=NUMBER)
        times[i, j] = np.sum(res) / REPEAT / NUMBER
        print(".", end="")
    print(s)

....1
....10
....100
....200
....500
....1000
....2000
....3000
....5000
....10000
....20000
....50000
....100000


In [20]:
l = [pd.DataFrame({'name': f.__name__, 
                   'x': sizes, 
                   'y': times[:, j]
                  }) 
     for j, f in enumerate(funcs)]
df = pd.concat(l)

In [222]:
fig = px.line(df, x='x', y='y', color='name', 
              log_x=True, log_y=True, markers=True, title='float64')
fig.show()

In [21]:
fig = px.line(df, x='x', y='y', color='name', 
              log_x=True, log_y=True, markers=True, title='float32')
fig.show()